In [74]:
from datetime import datetime
from datetime import timedelta
import requests
import json
import pandas as pd

In [58]:
#KEYS
app_key = ''
app_token = ''
url_api_vtex = ''
url_api_all_orders = url_api_vtex+'orders/'


In [105]:
def get_all_orders(beggining_date,ending_date,page=1,per_page=100):
  
  beggining_date =  beggining_date.strftime("%Y-%m-%d")+"T03:00:00.000Z"
  ending_date = ending_date.strftime("%Y-%m-%d")+"T02:59:59.999Z"
  
  #url of getting orders
  url = url_api_all_orders +"?"+"f_creationDate=creationDate:%5B{beggining_date}%20TO%20{ending_date}%5D&orderBy=creationDate,desc&per_page={per_page}&page={page}"
  url = url.format(beggining_date=beggining_date,ending_date=ending_date,per_page=per_page,page=page)  
  payload = {}
  headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'X-VTEX-API-AppToken':app_token ,
    'X-VTEX-API-AppKey': app_key
  }
  response = requests.request('GET', url, headers = headers, data = payload, allow_redirects=False)
  return json.loads(response.text)

In [93]:
def process_all_orders_day(start_day=datetime.now()):
    data = []
    end_day = start_day+timedelta(days=1)
    first_page_data = get_all_orders(start_day,end_day,1)
    total_pages = first_page_data["paging"]["pages"]
    currentPage = 1
    orders = first_page_data["list"]
    data.extend(orders)
    for page in range(2,total_pages+1):
        orders = get_all_orders(start_day,end_day,page)
        data.extend(orders["list"])
    return data

In [106]:
data = process_all_orders_day()

In [95]:
def data_to_dataframe(data):
    rows = []
    for order in data:
        row = []
        for key in order.keys():
           row.append(order[key])
        rows.append(row)
    return pd.DataFrame(rows,columns=order.keys())

In [102]:
df = data_to_dataframe(data)
